In [15]:
import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report,confusion_matrix

## Carregar os dados

In [2]:
df_leads = pd.read_csv('../bagging/df_airline_sat_cleaned.csv')

In [3]:
df_leads.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1


In [4]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  object 
 1   Customer Type                      103904 non-null  object 
 2   Age                                103904 non-null  int64  
 3   Type of Travel                     103904 non-null  object 
 4   Class                              103904 non-null  object 
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

## Preparação dos dados

In [6]:
X = df_leads.drop(columns=['satisfaction'], axis=1)
y = df_leads['satisfaction']

In [7]:
import joblib

preprocessor = joblib.load('../bagging/preprocessor.pkl')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=51)

# Aplicar preprocessor
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [10]:
print(f'Conjunto de Treinamento: {X_train.shape}')
print(f'Conjunto de Teste: {X_test.shape}')

Conjunto de Treinamento: (83123, 22)
Conjunto de Teste: (20781, 22)


In [11]:
preprocessor.get_feature_names_out()

array(['nom__Gender_Male', 'nom__Customer Type_disloyal Customer',
       'nom__Type of Travel_Personal Travel', 'ord_classe__Class',
       'notas__Inflight wifi service',
       'notas__Departure/Arrival time convenient',
       'notas__Ease of Online booking', 'notas__Gate location',
       'notas__Food and drink', 'notas__Online boarding',
       'notas__Seat comfort', 'notas__Inflight entertainment',
       'notas__On-board service', 'notas__Leg room service',
       'notas__Baggage handling', 'notas__Checkin service',
       'notas__Inflight service', 'notas__Cleanliness', 'num_cont__Age',
       'num_cont__Flight Distance',
       'num_cont__Departure Delay in Minutes',
       'num_cont__Arrival Delay in Minutes'], dtype=object)

## Treinamento do Modelo de Voting Classifier

In [12]:
# Criar o modelo de VotingClassifier
lr_model = LogisticRegression(random_state=51)

# Para executar o modelo do Soft voting, é necessário colocar o hiperparâmetro probability = True no modelo SVC
svc_model = SVC(probability=True,kernel='linear')
tree_model = DecisionTreeClassifier(random_state=51)

voting_model = VotingClassifier(
    estimators=[
        ('logistic regression', lr_model),
        ('svc', svc_model),
        ('decision tree',tree_model)
    ],

    # Hard faz a votação pela maioria das predições dos estimadores
    # Soft faz a votação pela média ponderada das probabilidades de cada classe vindas de cada estimador
    voting='soft'
)


In [13]:
voting_model.fit(X_train,y_train)

,"estimators estimators: list of (str, estimator) tuplesInvoking the ``fit`` method on the ``VotingClassifier`` will fit clonesof those original estimators that will be stored in the class attribute``self.estimators_``. An estimator can be set to ``'drop'`` using:meth:`set_params`... versionchanged:: 0.21 ``'drop'`` is accepted. Using None was deprecated in 0.22 and support was removed in 0.24.","[('logistic regression', ...), ('svc', ...), ...]"
,"voting voting: {'hard', 'soft'}, default='hard'If 'hard', uses predicted class labels for majority rule voting.Else if 'soft', predicts the class label based on the argmax ofthe sums of the predicted probabilities, which is recommended foran ensemble of well-calibrated classifiers.",'soft'
,"weights weights: array-like of shape (n_classifiers,), default=NoneSequence of weights (`float` or `int`) to weight the occurrences ofpredicted class labels (`hard` voting) or class probabilitiesbefore averaging (`soft` voting). Uses uniform weights if `None`.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for ``fit``.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionadded:: 0.18",None
,"flatten_transform flatten_transform: bool, default=TrueAffects shape of transform output only when voting='soft'If voting='soft' and flatten_transform=True, transform method returnsmatrix with shape (n_samples, n_classifiers * n_classes). Ifflatten_transform=False, it returns(n_classifiers, n_samples, n_classes).",True
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting will be printed as itis completed... versionadded:: 0.23",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001


## Análise dos Resultados

In [14]:
y_pred = voting_model.predict(X_test)

In [13]:
y_pred

array([1, 0, 0, ..., 0, 0, 1], shape=(1815,))

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93     11730
           1       0.93      0.88      0.90      9051

    accuracy                           0.92     20781
   macro avg       0.92      0.91      0.92     20781
weighted avg       0.92      0.92      0.92     20781



In [18]:
conf_matrix = confusion_matrix(y_test,y_pred)

fig = px.imshow(
    conf_matrix,
    labels=dict(x = 'Predição', y='Real', color='Contagem'),
    x=['neutral or dissatisfied', 'satisfied'],
    y=['neutral or dissatisfied', 'satisfied'],
    color_continuous_scale='Viridis'
)


fig.update_traces(text=conf_matrix,texttemplate="%{z}")
fig.update_layout(coloraxis_showscale=False)

fig.show()

In [19]:
# Carregar as importâncias dos modelos
importances = []

for estimator in voting_model.estimators_:

    # Para modelos lineares, retorna coef
    if hasattr(estimator, 'coef_'):
        importances.append(np.abs(estimator.coef_[0]))
    # Para modelos baseados em árvores
    elif hasattr(estimator,'feature_importances_'):
        importances.append(estimator.feature_importances_)
    else:
        print(f'Não foi possível carregar as importâncias do modelo {type(estimator).__name__}')

        

In [20]:
importancia_media = np.mean(importances,axis=0)

importancia_media

array([0.02659367, 1.15704996, 1.73629672, 0.11624914, 1.1088737 ,
       0.35868093, 0.36091236, 0.0554398 , 0.04094588, 1.89218505,
       0.1995952 , 0.1056642 , 0.8958171 , 0.69767757, 0.3066239 ,
       0.93036934, 0.38398938, 0.6234376 , 0.1102114 , 0.07415142,
       0.02626976, 0.06244521])

In [21]:
feature_names = preprocessor.get_feature_names_out()
feature_names

array(['nom__Gender_Male', 'nom__Customer Type_disloyal Customer',
       'nom__Type of Travel_Personal Travel', 'ord_classe__Class',
       'notas__Inflight wifi service',
       'notas__Departure/Arrival time convenient',
       'notas__Ease of Online booking', 'notas__Gate location',
       'notas__Food and drink', 'notas__Online boarding',
       'notas__Seat comfort', 'notas__Inflight entertainment',
       'notas__On-board service', 'notas__Leg room service',
       'notas__Baggage handling', 'notas__Checkin service',
       'notas__Inflight service', 'notas__Cleanliness', 'num_cont__Age',
       'num_cont__Flight Distance',
       'num_cont__Departure Delay in Minutes',
       'num_cont__Arrival Delay in Minutes'], dtype=object)

In [22]:
df_feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importancia_media})

df_feature_importances

,Feature,Importance
0,nom__Gender_Male,0.026594
1,nom__Customer Type_disloyal Customer,1.157050
2,nom__Type of Travel_Personal Travel,1.736297
3,ord_classe__Class,0.116249
4,notas__Inflight wifi service,1.108874
5,notas__Departure/Arrival time convenient,0.358681
6,notas__Ease of Online booking,0.360912
7,notas__Gate location,0.055440
8,notas__Food and drink,0.040946
9,notas__Online boarding,1.892185


In [23]:
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)
df_feature_importances

,Feature,Importance
20,num_cont__Departure Delay in Minutes,0.026270
0,nom__Gender_Male,0.026594
8,notas__Food and drink,0.040946
7,notas__Gate location,0.055440
21,num_cont__Arrival Delay in Minutes,0.062445
19,num_cont__Flight Distance,0.074151
11,notas__Inflight entertainment,0.105664
18,num_cont__Age,0.110211
3,ord_classe__Class,0.116249
10,notas__Seat comfort,0.199595


In [24]:
fig = px.bar(df_feature_importances,
             x='Importance',
             y='Feature',
             orientation='h',
             title='Importância das Features (Voting Classifier)'
)

fig.update_layout(height=1280, width=1000)
fig.show()